In [1]:
from dotenv import load_dotenv
import os
from groq import Groq


load_dotenv()

True

In [2]:
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

chat_completion = client.chat.completions.create(
    messages=[{"role": "user", "content": "What is an AI Agent?"}],
    model="llama3-8b-8192",
    temperature=0,
)

print(chat_completion.choices[0].message.content)

An AI agent is a software program that is designed to perform a specific task or set of tasks, and is capable of making decisions and taking actions based on its programming and the data it receives. AI agents can be thought of as autonomous entities that interact with their environment, other agents, and humans to achieve their goals.

There are several key characteristics that define an AI agent:

1. **Autonomy**: AI agents can operate independently, making decisions and taking actions without direct human intervention.
2. **Perception**: AI agents can perceive their environment through sensors, such as cameras, microphones, or other devices.
3. **Action**: AI agents can take actions in their environment, such as moving a robot arm or sending a message.
4. **Reasoning**: AI agents can reason about their environment and make decisions based on their programming and the data they receive.
5. **Learning**: AI agents can learn from their experiences and adapt to new situations.

There ar

In [3]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
            result = self.execute()
            self.messages.append({"role": "assistant", "content": result})
            return result
        else:
            print("NO MESSAGE")

    def execute(self):
        completion = client.chat.completions.create(
            model="llama3-70b-8192", messages=self.messages
        )
        return completion.choices[0].message.content

In [4]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
When you have a Thought you return the thought and Action
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate_total:
if amount = 200
then calculate_total(amount)

This returns amount * 1.2

Runs the calculate_total function and returns a JSON FORMAT output as follows:
{"result": 240, "fn": "calculate_total", "next": "PAUSE"}

get_product_price:
This uses the get_product_price function and passes in the value of the product
e.g. get_product_price('bike')

This uses the get_product_price with a product = 'bike', finds the price of the bike and then returns a JSON FORMAT output as follows:
{"result": 200, "fn": "get_product_price", "next": "PAUSE"}

Example session:


Question: What is the cost of a bike??
Thought: I need to find the cost of a bike|Action|get_product_price|bike
PAUSE 

You will be called again with this:

Observation|200

Thought: I need to calculate the total including the VAT|Action|calculate_total|200
PAUSE

You will be called again with this: 

Observation|240

If you have the answer, output it as the ANSWER.

ANSWER|The price of the bike including VAT is 240

Now it's your turn:
""".strip()


def calculate_total(amount) -> float:
    return int(amount * 1.2)


def get_product_price(product):
    if product == "bike":
        return 200
    if product == "tv":
        return 500
    if product == "laptop":
        return 600

    return None

In [5]:
agent = Agent(client=client, system=system_prompt)

In [6]:
result = agent("What is cost of a laptop including the VAT?")
# result = agent("What is cost of a bike including the VAT?")


print(result)

Thought: I need to find the cost of a laptop|Action|get_product_price|laptop


In [7]:
next = result.split("|")
print(next)
next_function = next[2]
next_arg = next[3]
print(next_function, next_arg)

['Thought: I need to find the cost of a laptop', 'Action', 'get_product_price', 'laptop']
get_product_price laptop


In [8]:
print(next_function, "|", next_arg)
if next_function == "get_product_price":
    result = get_product_price(next_arg)
    print(result)

get_product_price | laptop
600


In [9]:
next_prompt = f"Observation|{result}"
next_prompt

'Observation|600'

In [10]:
result = agent(next_prompt)
print(result)

Thought: I need to calculate the total including the VAT|Action|calculate_total|600


In [11]:
next = result.split("|")
print(next)
next_function = next[2]
next_arg = int(next[3])
print(f"FUNCTION - {next_function} |  ARGUMENT - {next_arg}")

['Thought: I need to calculate the total including the VAT', 'Action', 'calculate_total', '600']
FUNCTION - calculate_total |  ARGUMENT - 600


In [12]:
print(next_function, "|", next_arg)
if next_function == "calculate_total":
    result = calculate_total(next_arg)
    print(result)

calculate_total | 600
720


In [13]:
result = agent(next_prompt)
print(result)

Thought: I need to calculate the total including the VAT|Action|calculate_total|600
